# **Feature Extraction**

In [ ]:
def extract_w2v(df):
    X = []
    y = []

    for _, row in tqdm(df.iterrows(), total=len(df)):
        signal, sr = librosa.load(row["full_path"], sr=16000)

        inputs = processor(signal, sampling_rate=16000,
                           return_tensors="pt", padding=True)

        with torch.no_grad():
            outputs = wav2vec(inputs.input_values.to(device))
            hidden = outputs.last_hidden_state.squeeze(0)

        embedding = hidden.mean(dim=0)

        X.append(embedding.cpu().numpy())
        y.append(row["label"])

    return np.array(X), np.array(y)

In [ ]:
print("Extracting train embeddings...")
X_train_w2v, y_train_w2v = extract_w2v(train_df)

print("Extracting validation embeddings...")
X_val_w2v, y_val_w2v = extract_w2v(val_df)

Extracting train embeddings...


100%|██████████| 44896/44896 [25:49<00:00, 28.97it/s]


Extracting validation embeddings...


100%|██████████| 7988/7988 [04:37<00:00, 28.77it/s]


In [ ]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset_w2v = TensorDataset(
    torch.tensor(X_train_w2v, dtype=torch.float32),
    torch.tensor(y_train_w2v)
)

val_dataset_w2v = TensorDataset(
    torch.tensor(X_val_w2v, dtype=torch.float32),
    torch.tensor(y_val_w2v)
)

train_loader_w2v = DataLoader(train_dataset_w2v, batch_size=64, shuffle=True)
val_loader_w2v = DataLoader(val_dataset_w2v, batch_size=64)